Connected to spar (Python 3.11.9)

In [ ]:
def plot_hist(array):
    # Create the histogram
    fig, ax = plt.subplots(figsize=(12, 6))
    n, bins, patches = ax.hist(array, bins=50, edgecolor='black')

    # Add labels and title
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.set_title('Histogram of Data')

    # Function to format the label
    def format_label(bin_start, bin_end):
        return f'[{bin_start:.2f}, {bin_end:.2f})'

    # Add text labels for each bar
    for i, patch in enumerate(patches):
        width = patch.get_width()
        height = patch.get_height()
        x = patch.get_x()
        y = patch.get_y()
        label = format_label(bins[i], bins[i+1])
        
        # Rotate labels for better readability
        ax.text(x + width/2, y + height/2, label, 
                ha='center', va='center', rotation=90, fontsize=8)

    # Adjust layout to prevent clipping of labels
    plt.tight_layout()

    # Show the plot
    plt.show()

###########################

In [ ]:
first_layer_feat_idxes

NameError: name 'first_layer_feat_idxes' is not defined

In [ ]:
import argparse
import numpy as np
from math import prod
from src.enums import Measure
import os
import matplotlib.pyplot as plt


from dataclasses import dataclass

@dataclass
class Args:
    measure: Measure = Measure.pearson
    save: bool = False

In [ ]:
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--measure', type=Measure, help='interaction measure whose values will indicate the', default=Measure.pearson)
#     parser.add_argument('--truncate', type=bool, help='truncates the number of feature pairs that are sampled, used only for testing bc its fast', default=False)
#     parser.add_argument('--save', type=bool, help='whether to save results or not', default=False)
#     args = parser.parse_args()


args = Args()
print("Reading full interaction scores")
if args.measure == Measure.pearson:
    filename = "artefacts/similarity_measures/pearson_correlation/res_jb_sae_feature_similarity_pearson_correlation_1M_0.0_0.1.npz"
    with open(filename, 'rb') as data:
        interaction_scores = np.load(data)['arr_0']
elif args.measure == Measure.jaccard:
    filename = "artefacts/jaccard/jaccard_2024-07-02-a.npz"
    with open(filename, 'rb') as data:
        interaction_scores = np.load(data)['arr_0']
else:
    raise NotImplementedError(f"Haven't gotten around to {args.measure} yet")
num_layers, d_sae, _ = interaction_scores.shape

Reading full interaction scores


In [ ]:
flattened_per_layer_scores = interaction_scores.reshape(num_layers, d_sae**2)
# the indexes of the layer pairs, per layer, sorted by the score of each pair, descending
sorted_flattened_pairs_filename = "artefacts/sampled_interaction_measures/pearson_correlation/sorted_flattened_pair_idxes"
if os.path.exists(f"{sorted_flattened_pairs_filename}.npz"):
    sorted_desc_by_score_per_layer_pair_idxes = np.load(f"{sorted_flattened_pairs_filename}.npz")['arr_0']
else:
    print("Sorting each layer")
    # key line, very slow
    full_sorted_desc_by_score_per_layer_pair_idxes = np.argsort(flattened_per_layer_scores, axis=1)[:, ::-1]
    sorted_desc_by_score_per_layer_pair_idxes = full_sorted_desc_by_score_per_layer_pair_idxes[:, :500]
    np.savez(sorted_flattened_pairs_filename, sorted_desc_by_score_per_layer_pair_idxes)

In [ ]:
# parsing out the first layer and second layer idxes
first_layer_idx_arr = []
second_layer_idx_arr = []
for layer_idx in range(num_layers):
    first_layer_idxes, second_layer_idxes = np.unravel_index(sorted_desc_by_score_per_layer_pair_idxes[layer_idx], shape=(d_sae, d_sae))
    first_layer_idx_arr.append(first_layer_idxes)
    second_layer_idx_arr.append(second_layer_idxes)
first_layer_feat_idxes = np.array(first_layer_idx_arr)
second_layer_feat_idxes = np.array(second_layer_idx_arr)

In [ ]:
sorted_desc_by_score_per_layer_pair_idxes

array([[356978314, 356989137, 467889802, ..., 274670071, 527358606,
          4278495],
       [544132235, 558785663, 292799615, ...,  18498300, 169296199,
        483344726],
       [ 18134529, 481576948, 481572846, ...,  78913545, 178944816,
         12296020],
       ...,
       [170930692, 170948083, 435466756, ..., 270770834, 149557388,
        403819729],
       [113364127, 113348780, 540765343, ..., 600169404,  82671866,
        566237139],
       [583102731, 583098208, 482069506, ..., 131693941,  28856524,
        292975321]])

In [ ]:
per_layer_nonzero = np.count_nonzero(flattened_per_layer_scores, axis=1)

In [ ]:
per_layer_nonzero

array([172752, 165573, 165466, 197123, 238215, 271127, 303428, 334352,
       357107, 341060, 328678])

In [ ]:
sum(per_layer_nonzero)

2874881

In [ ]:
foo = {f"layer_{layer_idx}": full_sorted_desc_by_score_per_layer_pair_idxes[layer_idx, :per_layer_nonzero[layer_idx]] for layer_idx in range(num_layers)

SyntaxError: incomplete input (<ipython-input-11-6da43179aeb8>, line 1)

In [ ]:
foo = {f"layer_{layer_idx}": full_sorted_desc_by_score_per_layer_pair_idxes[layer_idx, :per_layer_nonzero[layer_idx]] for layer_idx in range(num_layers)}

NameError: name 'full_sorted_desc_by_score_per_layer_pair_idxes' is not defined

In [ ]:
flattened_per_layer_scores = interaction_scores.reshape(num_layers, d_sae**2)
# the indexes of the layer pairs, per layer, sorted by the score of each pair, descending
sorted_flattened_pairs_filename = "../bentefacts/pearson_correlation/sorted_nonzero_flattened_pair_idxes"
if os.path.exists(f"{sorted_flattened_pairs_filename}.npz"):
    sorted_desc_by_score_per_layer_pair_idxes = np.load(f"{sorted_flattened_pairs_filename}.npz")['arr_0']
else:
    print("Sorting each layer")
    # key line, very slow
    full_sorted_desc_by_score_per_layer_pair_idxes = np.argsort(flattened_per_layer_scores, axis=1)[:, ::-1]

Sorting each layer


In [ ]:
full_sorted_desc_by_score_per_layer_pair_idxes

array([[356978314, 356989137, 467889802, ..., 337315280, 337315087,
        337327719],
       [544132235, 558785663, 292799615, ..., 171457596, 233170840,
        543614872],
       [ 18134529, 481576948, 481572846, ..., 402626471, 402626472,
                0],
       ...,
       [170930692, 170948083, 435466756, ..., 402594201, 402594202,
                0],
       [113364127, 113348780, 540765343, ..., 295824851, 198705344,
        142740947],
       [583102731, 583098208, 482069506, ..., 402596938,         0,
         86993927]])

In [ ]:
per_layer_nonzero

array([172752, 165573, 165466, 197123, 238215, 271127, 303428, 334352,
       357107, 341060, 328678])

In [ ]:
foo = {f"layer_{layer_idx}": full_sorted_desc_by_score_per_layer_pair_idxes[layer_idx, :per_layer_nonzero[layer_idx]] for layer_idx in range(num_layers)}

In [ ]:
foo

{'layer_0': array([356978314, 356989137, 467889802, ..., 201326145, 201326267,
        201328435]),
 'layer_1': array([544132235, 558785663, 292799615, ..., 201280663, 201280664,
        201280893]),
 'layer_2': array([ 18134529, 481576948, 481572846, ..., 270518844, 596008934,
         45205612]),
 'layer_3': array([541109862, 531869286, 150573214, ..., 337232572,  38495638,
        201328376]),
 'layer_4': array([507216437, 507201373, 493241576, ..., 600590623, 356603447,
        239499072]),
 'layer_5': array([416538940, 416554820,  32800580, ..., 297735891, 201277417,
        201183361]),
 'layer_6': array([574429816, 184233129, 184224056, ...,  73156804, 252725618,
        103445359]),
 'layer_7': array([560074210, 367854386,  80094002, ..., 353027124, 579786888,
        262930092]),
 'layer_8': array([170930692, 170948083, 435466756, ..., 192830184, 482570451,
        158139827]),
 'layer_9': array([113364127, 113348780, 540765343, ..., 201255644, 201255643,
        201255645]),


In [ ]:
foo['layer_0'].shape

(172752,)

In [ ]:
# %% tb

import argparse
import numpy as np
from math import prod
from src.enums import Measure
import os
import matplotlib.pyplot as plt
from dataclasses import dataclass

@dataclass
class Args:
    measure: Measure = Measure.pearson
    save: bool = False

In [ ]:
# %%

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--measure', type=Measure, help='interaction measure whose values will indicate the', default=Measure.pearson)
#     parser.add_argument('--truncate', type=bool, help='truncates the number of feature pairs that are sampled, used only for testing bc its fast', default=False)
#     parser.add_argument('--save', type=bool, help='whether to save results or not', default=False)
#     args = parser.parse_args()


args = Args()
print("Reading full interaction scores")
if args.measure == Measure.pearson:
    filename = "artefacts/similarity_measures/pearson_correlation/res_jb_sae_feature_similarity_pearson_correlation_1M_0.0_0.1.npz"
    with open(filename, 'rb') as data:
        interaction_scores = np.load(data)['arr_0']
elif args.measure == Measure.jaccard:
    filename = "artefacts/jaccard/jaccard_2024-07-02-a.npz"
    with open(filename, 'rb') as data:
        interaction_scores = np.load(data)['arr_0']
else:
    raise NotImplementedError(f"Haven't gotten around to {args.measure} yet")
num_layers, d_sae, _ = interaction_scores.shape

Reading full interaction scores


In [ ]:
# %%

flattened_per_layer_scores = interaction_scores.reshape(num_layers, d_sae**2)
# the indexes of the layer pairs, per layer, sorted by the score of each pair, descending
sorted_flattened_pairs_filename = "../bentefacts/pearson_correlation/sorted_nonzero_flattened_pair_idxes"
if os.path.exists(f"{sorted_flattened_pairs_filename}.npz"):
    sorted_desc_by_score_per_layer_pair_idx_map = np.load(f"{sorted_flattened_pairs_filename}.npz")
else:
    print("Sorting each layer")
    # key line, very slow
    full_sorted_desc_by_score_per_layer_pair_idxes = np.argsort(flattened_per_layer_scores, axis=1)[:, ::-1]
    per_layer_nonzero = np.count_nonzero(full_sorted_desc_by_score_per_layer_pair_idxes, axis=1)
    only_nonzero_pair_idxes = {f"layer_{layer_idx}": full_sorted_desc_by_score_per_layer_pair_idxes[layer_idx, :per_layer_nonzero[layer_idx]] for layer_idx in range(num_layers)}

Sorting each layer


In [ ]:
only_nonzero_pair_idxes

{'layer_0': array([356978314, 356989137, 467889802, ..., 337329204, 337315280,
        337315087]),
 'layer_1': array([544132235, 558785663, 292799615, ..., 237914008, 171457596,
        233170840]),
 'layer_2': array([ 18134529, 481576948, 481572846, ..., 402626470, 402626471,
        402626472]),
 'layer_3': array([541109862, 531869286, 150573214, ..., 402620248, 402620247,
                0]),
 'layer_4': array([507216437, 507201373, 493241576, ..., 402613784, 402613785,
        402613786]),
 'layer_5': array([416538940, 416554820,  32800580, ..., 402607510,         0,
        238956925]),
 'layer_6': array([574429816, 184233129, 184224056, ..., 402602270, 402602271,
        402602272]),
 'layer_7': array([560074210, 367854386,  80094002, ..., 402598131, 402598132,
        402598133]),
 'layer_8': array([170930692, 170948083, 435466756, ..., 402594200, 402594201,
        402594202]),
 'layer_9': array([113364127, 113348780, 540765343, ...,         0, 295824851,
        198705344]),


In [ ]:
np.savez(sorted_flattened_pairs_filename, only_nonzero_pair_idxes)

FileNotFoundError: [Errno 2] No such file or directory: '../bentefacts/pearson_correlation/sorted_nonzero_flattened_pair_idxes.npz'

In [ ]:
np.savez(sorted_flattened_pairs_filename, only_nonzero_pair_idxes)

OverflowError: serializing a bytes object larger than 4 GiB requires pickle protocol 4 or higher

In [ ]:
np.savez_compressed(sorted_flattened_pairs_filename, only_nonzero_pair_idxes)

OverflowError: serializing a bytes object larger than 4 GiB requires pickle protocol 4 or higher

In [ ]:
np.savez_compressed(sorted_flattened_pairs_filename, only_nonzero_pair_idxes, pickle_kwargs={"protocol": 4})

OverflowError: serializing a bytes object larger than 4 GiB requires pickle protocol 4 or higher

In [ ]:
np.savez_compressed(sorted_flattened_pairs_filename, only_nonzero_pair_idxes, allow_pickle=True, pickle_kwargs={"protocol": 4})

OverflowError: serializing a bytes object larger than 4 GiB requires pickle protocol 4 or higher

In [ ]:
True

True

In [ ]:
np.savez_compressed(sorted_flattened_pairs_filename, **only_nonzero_pair_idxes)

KeyboardInterrupt: 

In [ ]:
per_layer_nonzero

array([603979775, 603979775, 603979775, 603979775, 603979775, 603979775,
       603979775, 603979775, 603979775, 603979775, 603979775])

In [ ]:
only_nonzero_pair_idxes.shape

AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
only_nonzero_pair_idxes['layer_0'].shape

(603979775,)

In [ ]:
    per_layer_nonzero = np.count_nonzero(interaction_scores, axis=1)

In [ ]:
per_layer_nonzero

array([[ 2,  1, 14, ...,  1,  1, 20],
       [19,  1,  2, ...,  6,  1,  2],
       [ 1,  0,  1, ...,  3,  5, 32],
       ...,
       [13,  5, 20, ...,  5, 19, 13],
       [15, 15, 16, ...,  3,  3, 14],
       [33, 15, 10, ..., 30,  0,  6]])

In [ ]:
    per_layer_nonzero = np.count_nonzero(flattened_per_layer_scores, axis=1)

In [ ]:
per_layer_nonzero

array([172752, 165573, 165466, 197123, 238215, 271127, 303428, 334352,
       357107, 341060, 328678])

In [ ]:
only_nonzero_pair_idxes = {f"layer_{layer_idx}": full_sorted_desc_by_score_per_layer_pair_idxes[layer_idx, :per_layer_nonzero[layer_idx]] for layer_idx in range(num_layers)}

In [ ]:
only_nonzero_pair_idxes

{'layer_0': array([356978314, 356989137, 467889802, ..., 201326145, 201326267,
        201328435]),
 'layer_1': array([544132235, 558785663, 292799615, ..., 201280663, 201280664,
        201280893]),
 'layer_2': array([ 18134529, 481576948, 481572846, ..., 270518844, 596008934,
         45205612]),
 'layer_3': array([541109862, 531869286, 150573214, ..., 337232572,  38495638,
        201328376]),
 'layer_4': array([507216437, 507201373, 493241576, ..., 600590623, 356603447,
        239499072]),
 'layer_5': array([416538940, 416554820,  32800580, ..., 297735891, 201277417,
        201183361]),
 'layer_6': array([574429816, 184233129, 184224056, ...,  73156804, 252725618,
        103445359]),
 'layer_7': array([560074210, 367854386,  80094002, ..., 353027124, 579786888,
        262930092]),
 'layer_8': array([170930692, 170948083, 435466756, ..., 192830184, 482570451,
        158139827]),
 'layer_9': array([113364127, 113348780, 540765343, ..., 201255644, 201255643,
        201255645]),


In [ ]:
only_nonzero_pair_idxes['layer_0'].shape

(172752,)

In [ ]:
np.savez_compressed(sorted_flattened_pairs_filename, only_nonzero_pair_idxes)

In [ ]:
    max_num_nonzeros = max(val for val in only_nonzero_pair_idxes)

In [ ]:
max_num_nonzeros

'layer_9'

In [ ]:
max_num_nonzeros = max(val for val in only_nonzero_pair_idxes.values())

ValueError: operands could not be broadcast together with shapes (165573,) (172752,) 

In [ ]:
max_num_nonzeros = max(len(val) for val in only_nonzero_pair_idxes.values())

In [ ]:
max_num_nonzeros

357107

In [ ]:
per_layer_nonzero

array([172752, 165573, 165466, 197123, 238215, 271127, 303428, 334352,
       357107, 341060, 328678])

In [ ]:
np.array(
    [
        np.pad(
            only_nonzero_pair_idxes[f'layer_{layer_idx}'].astype(float), 
            (0, max_num_nonzeros - len(row)), 
            mode='constant', 
            constant_values=np.nan
        ) 
        for layer_idx in len(num_layers)
    ]
)

TypeError: object of type 'int' has no len()

In [ ]:
np.array(
    [
        np.pad(
            only_nonzero_pair_idxes[f'layer_{layer_idx}'].astype(float), 
            (0, max_num_nonzeros - len(row)), 
            mode='constant', 
            constant_values=np.nan
        ) 
        for layer_idx in len(num_layers)
    ]
)

TypeError: object of type 'int' has no len()

In [ ]:
np.array(
    [
        np.pad(
            only_nonzero_pair_idxes[f'layer_{layer_idx}'].astype(float), 
            (0, max_num_nonzeros - len(only_nonzero_pair_idxes[f'layer_{layer_idx}'])), 
            mode='constant', 
            constant_values=np.nan
        ) 
        for layer_idx in range(num_layers)
    ]
)

array([[3.56978314e+08, 3.56989137e+08, 4.67889802e+08, ...,
                   nan,            nan,            nan],
       [5.44132235e+08, 5.58785663e+08, 2.92799615e+08, ...,
                   nan,            nan,            nan],
       [1.81345290e+07, 4.81576948e+08, 4.81572846e+08, ...,
                   nan,            nan,            nan],
       ...,
       [1.70930692e+08, 1.70948083e+08, 4.35466756e+08, ...,
        1.92830184e+08, 4.82570451e+08, 1.58139827e+08],
       [1.13364127e+08, 1.13348780e+08, 5.40765343e+08, ...,
                   nan,            nan,            nan],
       [5.83102731e+08, 5.83098208e+08, 4.82069506e+08, ...,
                   nan,            nan,            nan]])

In [ ]:
# %% tb

import argparse
import numpy as np
from math import prod
from src.enums import Measure
import os
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict
@dataclass
class Args:
    measure: Measure = Measure.pearson
    save: bool = False

In [ ]:
# %%

def convert_only_nonzero_pair_idxes_to_padded_arr(only_nonzero_pair_idxes: Dict):
    """turns a ragged dictionary of lists of indexes (all the nonzero pairs for that layer)
    into a nice rectangular matrix where shorter rows are padded to the length of the longest
    with nans

    Args:
        only_nonzero_pair_idxes (Dict): {"layer_idx": all_nonzero_flattened_pair_idxes_for_this_layer}
    """
    max_num_nonzeros = max(len(val) for val in only_nonzero_pair_idxes.values())
    return np.array(
        [
            np.pad(
                only_nonzero_pair_idxes[f'layer_{layer_idx}'].astype(float), 
                (0, max_num_nonzeros - len(only_nonzero_pair_idxes[f'layer_{layer_idx}'])), 
                mode='constant', 
                constant_values=np.nan
            ) 
            for layer_idx in range(num_layers)
        ]
    )

In [ ]:
    padded_nonzero_sorted_desc_by_score_per_layer_pair_idxes =convert_only_nonzero_pair_idxes_to_padded_arr(only_nonzero_pair_idxes)

In [ ]:
padded_nonzero_sorted_desc_by_score_per_layer_pair_idxes.shape

(11, 357107)